# NHN AI EasyMaker SDK 사용 예제

이 노트북에서는 NHN AI EasyMaker SDK를 사용하여 엔드포인트를 생성하고 관리하는 방법을 보여줍니다.

In [ ]:
# 필요한 라이브러리 임포트
import easymaker

In [ ]:
# 필요한 변수 설정
region="kr1"
appkey="YOUR_APP_KEY"

# 인증 토큰 발급 가이드 : https://docs.nhncloud.com/ko/nhncloud/ko/public-api/api-authentication/#_1
user_access_key_id="YOUR_ACCESS_KEY_ID"
secret_access_key="YOUR_SECRET_ACCESS_KEY"
access_token=easymaker.sample.get_access_token(user_access_key_id, secret_access_key),

## SDK 초기화

SDK를 사용하기 위해 초기화합니다.

In [ ]:
# SDK 초기화
easymaker.init(
    region=region,
    appkey=appkey,
    access_token=access_token,
)

## 엔드포인트 생성

모델 ID를 사용하여 엔드포인트를 생성합니다.

In [4]:
# 모델 생성
model = easymaker.Model().create_by_model_upload_uri(
    model_type_code=easymaker.sample.classification_model.model_type,
    model_upload_uri=easymaker.sample.classification_model.model_uri,
    model_name="easymaker_model_sample",
)

[AI EasyMaker] Model create request complete. model_id: g5t14lsibycv
[AI EasyMaker] Model create status: CREATE_REQUESTED (0:00:00) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:10) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:20) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:30) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:40) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:50) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:00) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:10) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:20) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:30) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:40) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRES

In [ ]:
# 인스턴스 목록 조회
instance_type_list = easymaker.Endpoint.get_instance_type_list()
for instance in instance_type_list:
    instance.print_info()
instance_type_name = instance_type_list[0].name


[파라미터]

| 이름                                                          | 타입      | 필수 여부 | 기본값   | 유효 범위                      | 설명                                                                     |
|-------------------------------------------------------------|---------|-------|-------|----------------------------|------------------------------------------------------------------------|
| endpoint_name                                               | String  | 필수    | 없음    | 최대 50자                     | 엔드포인트 이름                                                               |
| description                                        | String  | 선택    | 없음    | 최대 255자                    | 엔드포인트에 대한 설명                                                           |
| instance_type_name                                      | String  | 필수    | 없음    | 없음                         | 엔드포인트에 사용될 인스턴스 타입 이름                                                  |
| instance_count                                     | Integer | 선택    | 1     | 1~10                       | 엔드포인트에 사용될 인스턴스 수                                                      |
| endpoint_model_resource_list                                | Array   | 필수    | 없음    | 최대 10개                     | 스테이지에 사용될 리소스 정보                                                 |
| endpoint_model_resource_list[0].modelId                     | String   | 필수    | 없음    | 없음                       | 스테이지 리소스로 생성할 모델 ID                                   |
| endpoint_model_resource_list[0].resourceOptionDetail        | Object   | 필수    | 없음    |                                  | 스테이지 리소스의 상세 정보                 |
| endpoint_model_resource_list[0].resourceOptionDetail.cpu    | Double   | 필수    | 없음    | 0.0~                             | 스테이지 리소스에 사용될 CPU                |
| endpoint_model_resource_list[0].resourceOptionDetail.memory | Object   | 필수    | 없음    | 1Mi~                             | 스테이지 리소스에 사용될 메모리             |
| endpoint_model_resource_list[0].podAutoScaleEnable          | Boolean  | 선택    | False   | True, False                      | 스테이지 리소스에 사용될 파드 오토 스케일러 |
| endpoint_model_resource_list[0].scaleMetricCode             | String   | 선택    | 없음    | CPU_UTILIZATION, MEMORY_UTILIZATION | 스테이지 리소스에 사용될 증설 단위          |
| endpoint_model_resource_list[0].scaleMetricTarget           | Integer  | 선택    | 없음    | 1~                               | 스테이지 리소스에 사용될 증설 임계치 값     |
| endpoint_model_resource_list[0].description                 | String   | 선택    | 없음    | 최대 255자                  | 스테이지 리소스에 대한 설명                                       |
| use_log                                                     | Boolean | 선택    | False | True, False                | Log & Crash Search 서비스에 로그를 남길지 여부                                             |
| wait                                                        | Boolean | 선택    | True   | True, False | True: 생성이 완료된 이후 반환, False: 생성 요청 후 즉시 반환 |


In [2]:
# 엔드포인트 생성
endpoint = easymaker.Endpoint().create(
    endpoint_name="endpoint_sample",
    description="Easymaker sample endpoint",
    endpoint_model_resource_list=[
        easymaker.EndpointModelResource(
            model_id=model.model_id,
            pod_auto_scale_enable=True,
            scale_metric_code=easymaker.SCALE_METRIC_CODE.CPU_UTILIZATION,
            scale_metric_target=50,
            resource_option_detail=easymaker.ResourceOptionDetail(
                cpu="1.55",
                memory="3730Mi",
            ),
        )
    ],
    instance_type_name=instance_type_name,
    instance_count=1,
    autoscaler_enable=True,
    autoscaler_min_node_count=1,
    autoscaler_max_node_count=10,
    autoscaler_scale_down_enable=True,
    autoscaler_scale_down_util_threshold=50,
    autoscaler_scale_down_unneeded_time=10,
    autoscaler_scale_down_delay_after_add=10,
    use_log=True,
)
endpoint.print_info()

[AI EasyMaker] Endpoint create request complete. endpoint_id: ie8rg9d2qa0y
[AI EasyMaker] Endpoint create status: CREATE_REQUESTED (0:00:00) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:00:10) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:00:20) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:00:30) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:00:40) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:00:50) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:01:00) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:01:10) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:01:20) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:01:30) Please wait...
[AI EasyMaker] Endpoint create status: CREATE_IN_PROGRESS (0:01:40) Please wait...
[AI EasyMaker]

## 엔드포인트 조회

생성된 엔드포인트를 조회합니다.

In [5]:
# 목록 조회
endpoint_list = easymaker.Endpoint.get_list()
for endpoint in endpoint_list:
    endpoint.print_info()

{
    "appKey": "qV5VIb2S1hr33zTS",
    "createdDatetime": "2025-05-23T14:34:26+09:00",
    "endpointId": "ie8rg9d2qa0y",
    "endpointName": "endpoint_sample",
    "endpointStatusCode": "ACTIVE",
    "apigwAppKey": "yswXI6rptH8UqnBM",
    "apigwRegion": "KR1",
    "apigwServiceId": "t2vqeenpgo",
    "endpointStageList": [
        {
            "description": "Easymaker sample endpoint",
            "appKey": "qV5VIb2S1hr33zTS",
            "createdDatetime": "2025-05-23T14:34:26+09:00",
            "endpointStageId": "urvwtzhv06uq",
            "endpointId": "ie8rg9d2qa0y",
            "endpointStageStatusCode": "ACTIVE",
            "apigwStageId": "e164a145-f432-4a7e-8c0b-f26955ef482b",
            "logAndCrashAppKey": "V1eIQHSgTbwTga2n",
            "instanceType": {
                "id": "35a73b57-58a7-434d-aa08-5249aaa95b3e",
                "gpu": false
            },
            "isClusterLatest": true,
            "defaultStage": true,
            "endpointModelList": [
      

In [ ]:
# 단건 조회
endpoint = easymaker.Endpoint(endpoint_id=endpoint.endpoint_id)

In [1]:
import easymaker

region="kr1"
appkey="qV5VIb2S1hr33zTS"
user_access_key_id="I7XDyq6mxK4DKLUfWbFB"
secret_access_key="bed6kJEG3S7Qi5kx"

# SDK 초기화
easymaker.init(
    region=region,
    appkey=appkey,
    access_token=easymaker.sample.get_access_token(user_access_key_id, secret_access_key),
    profile="alpha",
)
endpoint = easymaker.Endpoint(endpoint_id="ie8rg9d2qa0y")
endpoint.print_info()

{
    "appKey": "qV5VIb2S1hr33zTS",
    "createdDatetime": "2025-05-23T14:34:26+09:00",
    "endpointId": "ie8rg9d2qa0y",
    "endpointName": "endpoint_sample",
    "endpointStatusCode": "ACTIVE",
    "apigwAppKey": "yswXI6rptH8UqnBM",
    "apigwRegion": "KR1",
    "apigwServiceId": "t2vqeenpgo",
    "endpointStageList": [
        {
            "description": "Easymaker sample endpoint",
            "appKey": "qV5VIb2S1hr33zTS",
            "createdDatetime": "2025-05-23T14:34:26+09:00",
            "endpointStageId": "urvwtzhv06uq",
            "endpointId": "ie8rg9d2qa0y",
            "endpointStageStatusCode": "ACTIVE",
            "apigwStageId": "e164a145-f432-4a7e-8c0b-f26955ef482b",
            "logAndCrashAppKey": "V1eIQHSgTbwTga2n",
            "instanceType": {
                "id": "35a73b57-58a7-434d-aa08-5249aaa95b3e",
                "gpu": false
            },
            "isClusterLatest": true,
            "defaultStage": true,
            "endpointModelList": [
      

In [2]:
input_data1 = [6.0, 3.4, 4.5, 1.6]
input_data2 = [3.2, 4.7, 3.3, 2.4]
result = endpoint.predict(
    model_id=endpoint.default_stage.endpoint_model_list[0].model_id,
    json={"instances": [input_data1, input_data2]},
)
print(result)

{'predictions': [[0.0, 1.03283944e-20, 1.0], [0.0, 6.27650225e-19, 1.0]]}


#

## 엔드포인트 삭제

생성된 엔드포인트를 삭제합니다.

In [ ]:
# 엔드포인트 삭제
endpoint.delete()